# Change Composition Using MDAnalysis
We will use the following outline:
1. Import all necessary packages
2. Randomly select a specified number of DPPC
3. Edit those DPPC, preferably using MDAnalysis
4. Export using mdanalysis.

In [14]:
import numpy as np
import MDAnalysis as mda

input_file = "dppc_bilayer.gro"
bilayer = mda.Universe(input_file)
bilayeratoms = bilayer.atoms
individualatoms = bilayeratoms.resids

# Randomly select 35 lipids
randomnumberselectiontest = np.random.randint(1,129,35)
rlist = randomnumberselectiontest.tolist()
sel = bilayer.select_atoms("resid 0")

for i in rlist:
    sel = sel + bilayer.select_atoms("resid %s" % i)

# Rename these lipids to DBPC
sel.resnames = "DBPC"

# Select all DPPC and W to transfer to output file
DPPW = bilayer.select_atoms("resname DPPC") + bilayer.select_atoms("resname W")

# Select the atoms from DBPC that actually belong to all DBPC molecules. IE, chop molecules by not selecting them.
rm = bilayer.select_atoms("resname DBPC and not (name C2A or name C3A or name C4A or name C2B or name C3B or name C4B)")

# Combine the unedited DPPC and W with the edited DBPC molecules. Then convert this to an atom list and write it to a gro file.
new = mda.Merge(rm, DPPW)
newatoms = new.atoms
newatoms.write(filename="NewComp", format="GRO")

In [24]:
import sys

infile = sys.argv[1]
outtop = sys.argv[2]
new = mda.Universe(infile)

# Count the number of each molecule.
numDBPC = new.select_atoms("resname DBPC")
numDPPC = new.select_atoms("resname DPPC")
numW = new.select_atoms("resname W")

#Account for the number of atoms in each molecule
ndb = len(np.unique(numDBPC.resids))
ndp = len(np.unique(numDPPC.resids))
nwat = len(np.unique(numW.resids))

#Comment out water if there isn't any
comm = ''
if counter3 == 0:
    comm = comm + ';'

top = open(outtop, 'w')
top.write('#include "martini_v2.1.itp"'+ '\n'
           '#include "dppc_single.itp"'+'\n'
           '#include "dbpc_single.itp"'+'\n'
           '\n'
          '[ system ]' +'\n'
          'MIXED BILAYER'+'\n'
          '\n'
          '[ molecules ]'+'\n'
          'DPPC %s' % ndp +'\n'
          'DBPC %s' % ndb +'\n'
          '%sW %s' % (comm, nwat) +'\n')
top.close()

ValueError: Cannot autodetect topology type for file '-f' (file extension could not be parsed).
           You can use 'Universe(topology, ..., topology_format=FORMAT)' to explicitly specify the format and
           override automatic detection. Known FORMATs are:
           ['XML', 'XPDB', 'PRMTOP', 'Permissive_PDB', u'CONFIG', 'TOP', 'PSF', 'PDBQT', 'MOL2', 'PQR', 'DMS', 'TPR', 'CRD', 'GMS', 'PARM7', 'GRO', 'XYZ', 'DATA', 'PDB', u'HISTORY']
           See http://docs.mdanalysis.org/documentation_pages/topology/init.html#supported-topology-formats
           For missing formats, raise an issue at http://issues.mdanalysis.org